# Background

The shape of outputs in integrated gradients suggests that IG may overestimate some attribution values due to saturated gradients at interpolated inputs. To confirm this, we run SplitIG (which cuts off the interpolated inputs if the gradients are saturated) and examine the level of agreement.

In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import torch
import pandas as pd
import numpy as np

from functools import partial
from typing import Optional

from transformer_lens.utils import get_act_name, get_device
from transformer_lens import ActivationCache, HookedTransformer, HookedTransformerConfig
from transformer_lens.hook_points import HookPoint

import seaborn as sns
import matplotlib.pyplot as plt

from utils import Task, TaskDataset, logit_diff_metric, run_from_layer_fn, plot_attn_comparison, plot_correlation
from split_ig import SplitLayerIntegratedGradients


## Set up Split IG

In [ ]:
def compute_layer_to_output_attributions_split_ig(
    original_input, 
    layer_input, 
    layer_baseline, 
    target_layer, 
    prev_layer, 
    metric, 
    metric_labels, 
    ratio=0.95
):
    n_samples = original_input.size(0)
    
    # Take the model starting from the target layer
    forward_fn = lambda x: run_from_layer_fn(model, original_input, prev_layer, x, metric, metric_labels)

    # Attribute to the target_layer's output
    split_ig = SplitLayerIntegratedGradients(forward_fn, target_layer, multiply_by_inputs=True)
    attributions = split_ig.attribute(inputs=layer_input,
                                    baselines=layer_baseline,
                                    internal_batch_size=n_samples,
                                    attribute_to_layer_input=False,
                                    return_convergence_delta=False)
    grads, outputs, steps, scaled_features = attributions

    # Calculate threshold for split integrated gradients
    baseline_out = forward_fn(layer_baseline)
    final_out = forward_fn(layer_input)
    threshold = baseline_out + ratio * (final_out - baseline_out)
    threshold_index = None
    for i in range(len(outputs)):
        if outputs[i] > threshold:
            threshold_index = i
            break

    # Grads have already been scaled and multiplied by inputs
    left_ig = grads[:threshold_index]
    right_ig = grads[threshold_index:]

    return left_ig.sum(dim=0,keepdim=True), right_ig.sum(dim=0,keepdim=True), threshold_index

In [4]:
def split_integrated_gradients(model: HookedTransformer, clean_tokens: torch.Tensor, clean_cache: ActivationCache, corrupted_cache: ActivationCache, metric: callable, metric_labels):
    n_samples = clean_tokens.size(0)
    
    # Gradient attribution for neurons in MLP layers
    mlp_results = torch.zeros(n_samples, model.cfg.n_layers, model.cfg.d_mlp)
    # Gradient attribution for attention heads
    attn_results = torch.zeros(n_samples, model.cfg.n_layers, model.cfg.n_heads)

    # Calculate integrated gradients for each layer
    for layer in range(model.cfg.n_layers):

        # Gradient attribution on heads
        hook_name = get_act_name("result", layer)
        target_layer = model.hook_dict[hook_name]
        prev_layer_hook = get_act_name("z", layer)
        prev_layer = model.hook_dict[prev_layer_hook]

        layer_clean_input = clean_cache[prev_layer_hook]
        layer_corrupt_input = corrupted_cache[prev_layer_hook]

        # Shape [batch, seq_len, d_head, d_model]
        left_ig, right_ig, _ = compute_layer_to_output_attributions_split_ig(
            clean_tokens, layer_corrupt_input, layer_clean_input, target_layer, prev_layer, metric, metric_labels)

        # Calculate attribution score based on mean over each embedding, for each token
        per_token_score = left_ig.mean(dim=3)
        score = per_token_score.mean(dim=1)
        attn_results[:, layer] = score

        # Gradient attribution on MLP neurons
        hook_name = get_act_name("post", layer)
        target_layer = model.hook_dict[hook_name]
        prev_layer_hook = get_act_name("mlp_in", layer)
        prev_layer = model.hook_dict[prev_layer_hook]

        layer_clean_input = clean_cache[prev_layer_hook]
        layer_corrupt_input = corrupted_cache[prev_layer_hook]
        
        # Shape [batch, seq_len, d_model]
        left_ig, right_ig, _ = compute_layer_to_output_attributions_split_ig(
            clean_tokens, layer_corrupt_input, layer_clean_input, target_layer, prev_layer, metric, metric_labels)
        score = left_ig.mean(dim=1)
        mlp_results[:, layer] = score

    return mlp_results, attn_results

# Split IG and IOI

In [5]:
torch.set_grad_enabled(False)

device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
ioi_dataset = TaskDataset(Task.IOI)
ioi_dataloader = ioi_dataset.to_dataloader(batch_size=1)

In [7]:
clean_input, corrupted_input, labels = next(iter(ioi_dataloader))

clean_tokens = model.to_tokens(clean_input)
corrupted_tokens = model.to_tokens(corrupted_input)

clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = logit_diff_metric(clean_logits, labels)
print(f"Clean logit difference: {clean_logit_diff}")

corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
corrupted_logit_diff = logit_diff_metric(corrupted_logits, labels)
print(f"Corrupted logit difference: {corrupted_logit_diff}")

Clean logit difference: tensor([2.3827], device='cuda:0')
Corrupted logit difference: tensor([-2.3610], device='cuda:0')


In [17]:
ioi_split_ig_mlp, ioi_split_ig_attn = split_integrated_gradients(model, clean_tokens, clean_cache, corrupted_cache, logit_diff_metric, labels)

torch.save(ioi_split_ig_mlp, "saved_results/ioi_split_ig_mlp.pt")
torch.save(ioi_split_ig_attn, "saved_results/ioi_split_ig_attn.pt")

('forward_fn', 'inputs', 'target_ind', 'additional_forward_args', 'get_outputs', 'scattered_inputs', 'scattered_input', 'layer_forward_hook', 'hooks', 'layers', 'layer_idx', 'layer', 'hook', 'output', 'grads')
Split IG attribution...
4
<class 'tuple'> 4
0
torch.Size([0, 18, 12, 768]) torch.Size([50, 18, 12, 768])
('forward_fn', 'inputs', 'target_ind', 'additional_forward_args', 'get_outputs', 'scattered_inputs', 'scattered_input', 'layer_forward_hook', 'hooks', 'layers', 'layer_idx', 'layer', 'hook', 'output', 'grads')
Split IG attribution...
4
<class 'tuple'> 4
0
torch.Size([0, 18, 3072]) torch.Size([50, 18, 3072])
('forward_fn', 'inputs', 'target_ind', 'additional_forward_args', 'get_outputs', 'scattered_inputs', 'scattered_input', 'layer_forward_hook', 'hooks', 'layers', 'layer_idx', 'layer', 'hook', 'output', 'grads')
Split IG attribution...
4
<class 'tuple'> 4
0
torch.Size([0, 18, 12, 768]) torch.Size([50, 18, 12, 768])
('forward_fn', 'inputs', 'target_ind', 'additional_forward_ar

In [26]:
ioi_split_ig_mlp = torch.load("saved_results/ioi_split_ig_mlp.pt")
ioi_split_ig_attn = torch.load("saved_results/ioi_split_ig_attn.pt")

ioi_ap_mlp = torch.load("saved_results/ioi_ap_mlp.pt")
ioi_ap_attn = torch.load("saved_results/ioi_ap_attn.pt")

In [27]:
plot_attn_comparison(ioi_split_ig_attn, ioi_ap_attn, Task.IOI, model)
plot_correlation(ioi_split_ig_mlp, ioi_ap_mlp, ioi_split_ig_attn, ioi_ap_attn, Task.IOI)

AssertionError: 